![](post.png)

<!-- metadata: title -->
# Shazam Playlist to Youtube Playlist

<!-- metadata: subtitle -->
> ### Play Shazam playlist from Youtube

<!-- metadata: keywords, type=array -->
**Keywords:**
  - shazam
  - playlist
  - youtube

<!-- metadata: categories, type=array -->
**Categories:**
  - ideas
  - shazam
  - youtube

<!-- metadata: -->
**Description:**
<!-- metadata: description -->
This is a simple idea of a utility app to play songs discovered by [Shazam](https://www.shazam.com/myshazam). The songs are played from youtube.

## Imports

In [1]:
import pandas as pd
from pytube import Search, YouTube
import gradio as gr
from gradio.utils import NamedString
import json

## Preview Data

In [2]:
shazamlibrary_df = pd.read_csv('shazamlibrary.csv', header=1)
shazamlibrary_df

,Index,TagTime,Title,Artist,URL,TrackKey
0,1,2024-06-13,Until I Found You (Em Beihold Version),Stephen Sanchez & Em Beihold,https://www.shazam.com/track/581828191/until-i...,581828191
1,2,2024-06-13,Capitals of Europe: Nordic Region,Katrina Holland,https://www.shazam.com/track/651866832/capital...,651866832
2,3,2024-06-13,Capitals of Europe: Nordic Region,Katrina Holland,https://www.shazam.com/track/651866832/capital...,651866832
3,4,2024-06-11,Ven Ven,Lotus Beatz,https://www.shazam.com/track/696779462/ven-ven,696779462
4,5,2024-05-30,Ava,Famy,https://www.shazam.com/track/127047942/ava,127047942
...,...,...,...,...,...,...
222,223,2018-03-10,Lose Yourself,Felax,https://www.shazam.com/track/340620006/lose-yo...,340620006
223,224,2018-03-08,This Is What It Feels Like (feat. Trevor Guthrie),Armin van Buuren,https://www.shazam.com/track/89020928/this-is-...,83795151
224,225,2018-03-04,Roulette,Katy Perry,https://www.shazam.com/track/358615884/roulette,358615884
225,226,2018-03-04,Roulette,Katy Perry,https://www.shazam.com/track/358615884/roulette,358615884


## Remove Duplicates and Select `Title` and `Artist`.

In [3]:
shazamlibrary_df = shazamlibrary_df.drop_duplicates(subset=['TrackKey'])[['Title', 'Artist']]
shazamlibrary_df

,Title,Artist
0,Until I Found You (Em Beihold Version),Stephen Sanchez & Em Beihold
1,Capitals of Europe: Nordic Region,Katrina Holland
3,Ven Ven,Lotus Beatz
4,Ava,Famy
5,Hero,Bryan Todd Feat. Ashley Argota
...,...,...
221,恋人心,Wei Xin Yu
222,Lose Yourself,Felax
223,This Is What It Feels Like (feat. Trevor Guthrie),Armin van Buuren
224,Roulette,Katy Perry


In [4]:
def get_youtube_song(title: str, artist: str) -> YouTube | None:
    search_result = Search(f'{title} by {artist}')
    return search_result.results[0] if search_result.results else None

In [5]:
search_title, search_artist = shazamlibrary_df.loc[10].values
youtube: YouTube = get_youtube_song(search_title, search_artist)
youtube.watch_url

Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: Stay With Me by CHANYEOL & Punch
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.
Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: Stay With Me by CHANYEOL & Punch
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.


'https://youtube.com/watch?v=A8syhfGRx4A'

## Code to run the player

In [6]:
"""Shazam Playlist to Youtube Playlist"""

from typing import Optional
import pandas as pd
from pytube import Search, YouTube
from flask import Flask, request, send_from_directory

app = Flask(__name__)

@app.route('/')
def index():
    """Route handler for the home page"""
    try:
        return send_from_directory('.', 'index.html')
    except Exception as e:
        return str(e)
    
@app.route('/video_id', methods=['POST'])
def video_id() -> str:
    """Route handler for retrieving the YouTube video ID"""
    try:
        title: str = request.json.get('title')
        artist: str = request.json.get('artist')
        youtube: YouTube = _get_youtube_song(title, artist)
        return youtube.video_id
    except Exception as e:
        return str(e)

@app.route('/parse_csv', methods=['POST'])
def parse_csv():
    """Route handler for parsing the uploaded CSV file"""
    try:
        file = request.files['file']
        # Process the uploaded file
        shazamlibrary_df = pd.read_csv(file, header=1)
        shazamlibrary_df = shazamlibrary_df.drop_duplicates(subset=['TrackKey'])[['Title', 'Artist']]
        return shazamlibrary_df.to_json(orient="records")
    except Exception as e:
        return str(e)

def _get_youtube_song(title: str, artist: str) -> Optional[YouTube]:
    """Searches for a YouTube video based on the given title and artist"""
    search_result = Search(f'{title} by {artist}')
    return search_result.results[0] if search_result.results else None

## Test the App

> <https://toknow-ai-shazam-playlist-to-youtube-playlist.hf.space>

In [1]:
%%HTML

<iframe
	src="https://toknow-ai-shazam-playlist-to-youtube-playlist.hf.space"
	frameborder="0"
	width="100%"
	height="600"
	style="max-wdth:850px;width:100%"
></iframe>